In [2]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
import os

print("="*60)
print("MNIST DIGIT RECOGNITION - CUSTOM IMAGE PREDICTOR")
print("="*60)

# ============================================
# 1. LOAD AND PREPROCESS MNIST DATASET
# ============================================
print("\n[1/5] Loading MNIST dataset...")
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(f"✓ Training samples: {x_train.shape[0]}")
print(f"✓ Test samples: {x_test.shape[0]}")

# ============================================
# 2. BUILD CNN MODEL
# ============================================
print("\n[2/5] Building CNN model...")
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(learning_rate=0.01),
    metrics=['accuracy']
)
print("✓ Model architecture created")

# ============================================
# 3. TRAIN MODEL
# ============================================
print("\n[3/5] Training model (this may take 2-5 minutes)...")
history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=10,
    verbose=1,
    validation_data=(x_test, y_test)
)

test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\n✓ Training complete!")
print(f"✓ Test Accuracy: {test_accuracy*100:.2f}%")

# ============================================
# 4. SAVE MODEL
# ============================================
print("\n[4/5] Saving trained model...")
model.save('digit_recognition_model.h5')
print("✓ Model saved as 'digit_recognition_model.h5'")

# ============================================
# 5. CUSTOM IMAGE PREDICTION FUNCTION
# ============================================
print("\n[5/5] Setting up custom image predictor...")

def preprocess_custom_image(image_path):
    """
    Preprocess a custom image to match MNIST format
    - Converts to grayscale
    - Resizes to 28x28
    - Inverts if needed (MNIST has white digits on black background)
    - Normalizes pixel values
    """
    try:
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not read image from {image_path}")
        
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Resize to 28x28
        resized = cv2.resize(gray, (28, 28), interpolation=cv2.INTER_AREA)
        
        # Check if we need to invert (MNIST has white on black)
        # If the image has dark digits on white background, invert it
        if np.mean(resized) > 127:
            resized = 255 - resized
        
        # Normalize to 0-1 range
        normalized = resized.astype('float32') / 255.0
        
        # Reshape for model input
        processed = normalized.reshape(1, 28, 28, 1)
        
        return processed, resized
    
    except Exception as e:
        print(f"Error processing image: {e}")
        return None, None

def predict_custom_digit(image_path, show_visualization=True):
    """
    Predict digit from a custom image file
    
    Parameters:
    - image_path: Path to the image file
    - show_visualization: Whether to display the result
    
    Returns:
    - predicted_digit: The predicted digit (0-9)
    - confidence: Confidence percentage
    """
    print(f"\n{'='*60}")
    print(f"PREDICTING DIGIT FROM: {image_path}")
    print(f"{'='*60}")
    
    # Check if file exists
    if not os.path.exists(image_path):
        print(f"❌ Error: File '{image_path}' not found!")
        return None, None
    
    # Preprocess image
    processed_image, display_image = preprocess_custom_image(image_path)
    
    if processed_image is None:
        return None, None
    
    # Make prediction
    prediction = model.predict(processed_image, verbose=0)
    predicted_digit = np.argmax(prediction)
    confidence = np.max(prediction) * 100
    
    # Display results
    print(f"\n🎯 PREDICTED DIGIT: {predicted_digit}")
    print(f"📊 CONFIDENCE: {confidence:.2f}%")
    print(f"\nProbability Distribution:")
    for digit in range(10):
        prob = prediction[0][digit] * 100
        bar = '█' * int(prob / 2)
        print(f"  {digit}: {bar} {prob:.2f}%")
    
    if show_visualization:
        # Visualize
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        
        # Original preprocessed image
        axes[0].imshow(display_image, cmap='gray')
        axes[0].set_title('Preprocessed Image\n(28x28 grayscale)', fontsize=10)
        axes[0].axis('off')
        
        # Prediction probabilities
        axes[1].bar(range(10), prediction[0], color='steelblue')
        axes[1].set_xlabel('Digit')
        axes[1].set_ylabel('Probability')
        axes[1].set_title('Prediction Probabilities', fontsize=10)
        axes[1].set_xticks(range(10))
        axes[1].grid(axis='y', alpha=0.3)
        
        # Result display
        axes[2].text(0.5, 0.6, str(predicted_digit), 
                     fontsize=100, ha='center', va='center',
                     color='green' if confidence > 90 else 'orange',
                     weight='bold')
        axes[2].text(0.5, 0.2, f'{confidence:.1f}% confident', 
                     fontsize=14, ha='center', va='center')
        axes[2].set_xlim(0, 1)
        axes[2].set_ylim(0, 1)
        axes[2].axis('off')
        axes[2].set_title('Predicted Digit', fontsize=10)
        
        plt.tight_layout()
        plt.show()
    
    return predicted_digit, confidence

# ============================================
# EXAMPLE USAGE
# ============================================
print("\n" + "="*60)
print("✅ SETUP COMPLETE! Ready to predict custom digits!")
print("="*60)

print("\n📝 HOW TO USE:")
print("-" * 60)
print("1. Save your digit image (PNG, JPG, etc.)")
print("2. Use: predict_custom_digit('path/to/your/image.png')")
print("\n💡 TIPS FOR BEST RESULTS:")
print("   • Use clear, centered digits")
print("   • Black or white background works best")
print("   • Single digit per image")
print("   • Higher resolution is better (will be resized to 28x28)")
print("\n📌 EXAMPLE:")
print("   predict_custom_digit('my_digit.png')")
print("   predict_custom_digit('digit_5.jpg')")
print("-" * 60)

# Test with a random MNIST test image to demonstrate
print("\n🔍 DEMO: Testing with a sample from MNIST test set...")
sample_idx = np.random.randint(0, len(x_test))
sample_image = x_test[sample_idx].reshape(28, 28)
sample_label = np.argmax(y_test[sample_idx])

# Save sample as temporary file for demo
temp_image = (sample_image * 255).astype(np.uint8)
cv2.imwrite('digit.png', temp_image)

print(f"True label: {sample_label}")
predict_custom_digit('digit.png', show_visualization=True)

# Clean up temp file
if os.path.exists('digit.png'):
    os.remove('digit.png')

print("\n" + "="*60)
print("🎉 Ready to recognize your custom digit images!")
print("="*60)

ModuleNotFoundError: No module named 'cv2'